# Getting Started

## Introduction

This example describes how to connect to Granta MI and perform a basic query for impacted substances. It also
demonstrates how to view logging messages returned by the Granta MI server. For more details about the results of the
queries, see the examples in the [Impacted Substances](1_Impacted_Substances_Queries/index.rst) and
[Compliance](2_Compliance_Queries/index.rst) sections.

## Connect to Granta MI

First, use the ``ansys.grantami.bomanalytics.Connection`` class to connect to the Granta MI server. The ``Connection``
class uses a fluent interface to build the connection, which is always invoked in the following sequence:

1. Specify your Granta MI Service Layer URL as a parameter to the ``Connection`` class.
2. Specify the authentication method using a ``Connection.with_...()`` method.
3. Use the ``Connection.connect()`` method to finalize the connection. This returns a connection object, which is
   called ``cxn`` in these examples.

In [ ]:
from ansys.grantami.bomanalytics import Connection

server_url = "http://my_grantami_server/mi_servicelayer"

If you are running your Python script on Windows, you are generally able to use ``.with_autologon()``.

In [ ]:
cxn = Connection(server_url).with_autologon().connect()
cxn

If the Python script is running on Linux without Kerberos enabled, or you wish to use an account other than your
logged-in account, you can specify credentials explicitly.

In [ ]:
cxn = Connection(server_url).with_credentials("my_username", "my_password").connect()
cxn

OIDC and anonymous authentication methods are also available, but are beyond the scope of this example. See
the [ansys-openapi-common](https://github.com/pyansys/openapi-common) package documentation for more details.

## Construct a Query

Queries are also constructed using a fluent interface. However, the ``Query`` constructor takes no arguments. All
query details are specified using ``Query`` methods. To demonstrate this, we will build a query to
determine all substances present in an ABS material that are impacted by the REACH Candidate List legislation.

First import the ``queries`` module and create a ``MaterialImpactedSubstancesQuery`` object.

In [ ]:
from ansys.grantami.bomanalytics import queries

query = queries.MaterialImpactedSubstancesQuery()
query

Now add the material you want to query, by specifying its Material ID. (Alternate methods of specifying records
are shown in other examples.)

In [ ]:
query = query.with_material_ids(["plastic-abs-high-impact"])
query

Note that because the ``MaterialImpactedSubstancesQuery`` object has a fluent interface, we receive the same object
back that we started with, but with the Material IDs added.

Finally, add the legislation to the query.

In [ ]:
query = query.with_legislations(["REACH - The Candidate List"])
query

Fluent interfaces are designed to allow a complex object to be constructed in a single line of code. As such, we can
consolidate the cells above into a single step:

In [ ]:
query = queries.MaterialImpactedSubstancesQuery().with_material_ids(["plastic-abs-high-impact"]).with_legislations(["REACH - The Candidate List"])  # noqa: E501
query

Since the fluent interface can produce very long lines of code, it's necessary to break your query creation code into
multiple lines. The following multi-line format is used throughout the examples, and is functionally equivalent to
the cell above:

In [ ]:
query = (
    queries.MaterialImpactedSubstancesQuery()
    .with_material_ids(["plastic-abs-high-impact"])
    .with_legislations(["REACH - The Candidate List"])
)
query

The multi-line format is the recommended way of creating queries using this API.

## Run a Query

Now that we have our ``cxn`` and ``query`` objects, we can use the ``cxn.run()`` method to run the query. This returns
an object that contains the results of the query.

In [ ]:
result = cxn.run(query)
result

## Query Results

In the case of ``MaterialsImpactedSubstancesQuery``, the results object contains the list of substances present in
the material that are impacted by the specified legislations.

In [ ]:
result.impacted_substances

## Logging

All query results also contain a list of messages returned by the server whilst running the query. These are
sorted in order of decreasing severity. The same messages are also available in the MI Service Layer log file.

In [ ]:
result.messages

These messages are also available via the standard ``logging`` module using the ``ansys.grantami.bomanalytics``
logger. Alternatively, you can omit the logger name to get the root logger, which will include messages logged by all
packages.

The code below creates a log handler that outputs all 'ansys.grantami.bomanalytics' logger messages with severity INFO
and above to either the terminal or the notebook.

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ansys.grantami.bomanalytics")

result = cxn.run(query)